In [ ]:
# Creates a parquet file with trajectories from raw AIS data

from make_trajectories_from_AIS import ais_to_trajectory

filename = '../../data/raw/AIS_04-09_2022_stavanger/ais_202207.csv'  # read file from
size = -1  # set the number of AIS messages for processing (-1 processes the entire file)
start = 0  # index of the first message to process
save_to = '../../data/processed/202207_points_stavanger_cleaned_meta_full_dualSplit_2.parquet'  # save file to

ais_to_trajectory(filename, size, start, save_to)